# 🎭 Avatarify with Auto-Refresh Ngrok

**Never worry about disappearing tunnels again!**

This enhanced version:
1. Automatically refreshes ngrok tunnels every 60 minutes
2. Maintains stable connections without manual intervention
3. Provides real-time status updates
4. Self-heals from disconnections

Just run all cells sequentially and keep the notebook open during your session!

## 1. Install Dependencies

In [ ]:
# Clean environment
%cd /content
!rm -rf avatarify

# Clone Avatarify
!git clone -q https://github.com/alievk/avatarify.git
%cd avatarify

# Install dependencies
!git clone -q https://github.com/alievk/first-order-model.git fomm
!pip install -q face-alignment==1.0.0 msgpack_numpy pyyaml==6.0.1 pyngrok
!sudo apt-get -y install libgl1-mesa-glx > /dev/null

# Download models
!scripts/download_data.sh > /dev/null

print("✅ Dependencies installed")

In [ ]:
# 👇 Patch face_alignment/utils.py directly
!sed -i 's/t\[0, 0\] = resolution \/ h/t[0, 0] = float(resolution \/ h)/' /usr/local/lib/python*/dist-packages/face_alignment/utils.py
!sed -i 's/t\[1, 1\] = resolution \/ h/t[1, 1] = float(resolution \/ h)/' /usr/local/lib/python*/dist-packages/face_alignment/utils.py


In [ ]:
# Show patched lines in face_alignment/utils.py
!sed -n '75,90p' /usr/local/lib/python*/dist-packages/face_alignment/utils.py


## 2. Configure Ngrok

1. Get your ngrok authtoken from [https://dashboard.ngrok.com/auth](https://dashboard.ngrok.com/auth)
2. Choose your region:

In [ ]:
#@title Enter Ngrok Configuration

authtoken = "" #@param {type:"string"}
region = "us" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"]

print(f"✔ Ngrok configured: Region={region}")

## 3. Auto-Refresh System Setup

In [ ]:
from pyngrok import ngrok, conf
import threading
import time
from subprocess import Popen
import shlex
from IPython.display import clear_output

# ===== CONFIGURATION =====
TUNNEL_REFRESH_INTERVAL = 3600  # 60 minutes
current_in_addr = ""
current_out_addr = ""
worker_process = None
in_tunnel = None
out_tunnel = None
stop_event = threading.Event()

# ===== CORE FUNCTIONS =====
def create_tunnels():
    global in_tunnel, out_tunnel, current_in_addr, current_out_addr

    # Close existing tunnels
    if in_tunnel:
        try:
            ngrok.disconnect(in_tunnel.public_url)
            print(f"🔒 Closed input tunnel")
        except Exception as e:
            print(f"⚠️ Error closing input tunnel: {str(e)}")

    if out_tunnel:
        try:
            ngrok.disconnect(out_tunnel.public_url)
            print(f"🔒 Closed output tunnel")
        except Exception as e:
            print(f"⚠️ Error closing output tunnel: {str(e)}")

    # Create new tunnels
    print("🚇 Creating new tunnels...")
    in_tunnel = ngrok.connect(5557, "tcp", "avatarify-input")
    out_tunnel = ngrok.connect(5558, "tcp", "avatarify-output")

    current_in_addr = in_tunnel.public_url.replace("tcp://", "")
    current_out_addr = out_tunnel.public_url.replace("tcp://", "")

    print(f"✅ NEW INPUT: {current_in_addr}")
    print(f"✅ NEW OUTPUT: {current_out_addr}")
    return current_in_addr, current_out_addr

def start_worker(in_addr, out_addr):
    global worker_process

    # Stop existing worker
    if worker_process:
        try:
            worker_process.terminate()
            worker_process.wait(timeout=5)
            print("🔒 Stopped previous worker")
        except Exception as e:
            print(f"⚠️ Error stopping worker: {str(e)}")
            !pkill -f cam_fomm.py

    # Start new worker
    cmd = f'./run.sh --is-worker --in-addr tcp://{in_addr} --out-addr tcp://{out_addr} --no-vcam --no-conda'
    print(f"🚀 Starting worker: {cmd}")

    with open('/content/worker.log', 'w') as log_file:
        worker_process = Popen(shlex.split(cmd), stdout=log_file, stderr=log_file)

    # Wait for initialization
    time.sleep(8)
    print("✅ Worker started successfully")
    return cmd

def tunnel_manager():
    while not stop_event.is_set():
        try:
            in_addr, out_addr = create_tunnels()
            worker_cmd = start_worker(in_addr, out_addr)
            print(f"\n🔥 NEW CLIENT COMMAND:\n./run_mac.sh --is-client --in-addr tcp://{in_addr} --out-addr tcp://{out_addr}")

            # Wait until next refresh or stop signal
            for _ in range(TUNNEL_REFRESH_INTERVAL // 10):
                if stop_event.is_set():
                    return
                time.sleep(10)

        except Exception as e:
            print(f"⚠️ Tunnel manager error: {str(e)}")
            time.sleep(30)

# ===== INITIAL SETUP =====
print("🚀 Starting auto-refresh system...")
conf.get_default().auth_token = authtoken
conf.get_default().region = region

# Initial tunnel and worker setup
in_addr, out_addr = create_tunnels()
worker_cmd = start_worker(in_addr, out_addr)
print(f"\n🔥 INITIAL CLIENT COMMAND:\n./run_mac.sh --is-client --in-addr tcp://{in_addr} --out-addr tcp://{out_addr}")

# Start background refresh
manager_thread = threading.Thread(target=tunnel_manager, daemon=True)
manager_thread.start()

print("\n🔒 Tunnels will auto-refresh every 60 minutes")
print("💡 Keep this notebook running during your session")
print("ℹ️ Latest command always appears above this message")

## 4. Current Connection Command

In [ ]:
from IPython.display import Markdown

if current_in_addr and current_out_addr:
    command = f"./run_mac.sh --is-client --in-addr tcp://{current_in_addr} --out-addr tcp://{current_out_addr}"
    display(Markdown(f"### ✅ Active Connection Command:\n```\n{command}\n```"))
    print("Run this on your local machine")
else:
    print("⚠️ No active tunnels found. Run section 3 first.")

## 5. Real-time Monitoring

In [ ]:
#@title Live System Dashboard

import time
from datetime import datetime
from IPython.display import display, clear_output, HTML

print("🔄 Starting live dashboard - Press Stop when done")

try:
    while True:
        clear_output(wait=True)

        # System status
        display(HTML(f"<h3>🕒 {datetime.now().strftime('%H:%M:%S')} | Auto-Refresh System</h3>"))

        # Tunnel status
        if current_in_addr and current_out_addr:
            display(HTML(f"<p>🔌 <b>Input Tunnel:</b> {current_in_addr}</p>"))
            display(HTML(f"<p>🔌 <b>Output Tunnel:</b> {current_out_addr}</p>"))
        else:
            display(HTML("<p>⚠️ <b>No active tunnels</b></p>"))

        # Worker status
        worker_running = False
        if worker_process:
            worker_running = worker_process.poll() is None

        status = "✅ Running" if worker_running else "❌ Stopped"
        display(HTML(f"<p>🤖 <b>Worker Status:</b> {status}</p>"))

        # Latest logs
        display(HTML("<h4>📜 Latest Worker Logs:</h4>"))
        !tail -n 8 /content/worker.log 2>/dev/null || echo "No logs available"

        # Refresh reminder
        display(HTML("<p>\n🔁 Dashboard refreshes every 10 seconds</p>"))
        display(HTML("<p>🛑 Press the <b>Stop</b> button above to exit</p>"))

        time.sleep(10)
except:
    print("Dashboard stopped")

## 6. Cleanup (Run When Finished)

In [ ]:
print("🛑 Stopping session...")

# Signal manager thread to stop
if 'stop_event' in globals():
    stop_event.set()
    print("🔒 Stopping auto-refresh manager")

    # Wait for manager to exit
    if 'manager_thread' in globals():
        manager_thread.join(timeout=10)
        print("✅ Manager thread stopped")

# Close tunnels
try:
    if 'in_tunnel' in globals() and in_tunnel:
        ngrok.disconnect(in_tunnel.public_url)
        print("🔒 Input tunnel closed")
    if 'out_tunnel' in globals() and out_tunnel:
        ngrok.disconnect(out_tunnel.public_url)
        print("🔒 Output tunnel closed")
except Exception as e:
    print(f"⚠️ Error closing tunnels: {str(e)}")

# Stop worker
try:
    if 'worker_process' in globals() and worker_process:
        worker_process.terminate()
        worker_process.wait(timeout=5)
        print("🔒 Worker process stopped")
except Exception as e:
    print(f"⚠️ Error stopping worker: {str(e)}")
    !pkill -f cam_fomm.py

print("✅ Session ended safely")